# Imports

In [ ]:
import scraping_functions as sf
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Parameters

In [ ]:
link = "https://secure.tiktok.biz/results/list/sydneyhalfmarathon/"
dataBody = {'__EVENTTARGET': 'ctl00$pagePlaceHolder$btnNextT', '__VIEWSTATE': ''}
distance = "/21KM/"
nMax = 300 # Numbers of pages read

# Scraping

In [ ]:

for i in range(2001, 2018):

    url = link + str(i) + distance
    fullList = []
    k = 0
    goToNextPage = True
    
    while(goToNextPage):
        if (k == 0): 
            #Perform GET
            r = requests.get(url)
            
        else:
            # Perform POST
            r = requests.post(url, data = dataBody)

        # Parsing of useful data
        currentStatus = r.status_code
        if (currentStatus == 200):
            
            # Definition of the soup variables
            soup = BeautifulSoup(r.content, 'html.parser')
            state = soup.find(id="__VIEWSTATE")['value']
            button = soup.find(id='ctl00_pagePlaceHolder_btnNextT')
            tableSoup = soup.find('table')

            # Definition of the POST variables
            noNextButton = button.has_attr('disabled')
            dataBody['__VIEWSTATE'] = state
            
            # We use the parsing function
            tableContent = sf.HTMLtableToDic(tableSoup)
            fullList.extend(tableContent)
            #print (tableContent)
            print ("Year {} - Page {}".format(i,k+1))
            
            if(noNextButton):
                goToNextPage = False

        else:
            goToNextPage = False
            print ("Last HTTP request has failed")

                    
        if(k >= nMax - 1):
            goToNextPage = False
            print ("Number of pages maximum defined as {} is reached".format(nMax))

        k = k + 1

    print(len(fullList))
        
    df = pd.DataFrame(fullList)
    df = df.iloc[:,:-1] # Drop the last column (empty)
    print(df.tail())
    
    df.to_csv('data/output_{}.csv'.format(i), index=False)